In [1]:
import sys
sys.path.append('../../')
import django,os,glob
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "dva.settings")
django.setup()

In [2]:
import tensorflow as tf

In [3]:
input_path = "/Users/aub3/media/1/frames/*.jpg"
model_path = "/Users/aub3/Dropbox/DeepVideoAnalytics/dvalib/object_detection/ssd_mobilenet_v1_coco_11_06_2017/frozen_inference_graph.pb"
detection_graph = tf.Graph()
with detection_graph.as_default():
    filename_queue = tf.train.string_input_producer(tf.train.match_filenames_once(input_path))
    image_reader = tf.WholeFileReader()
    imname, image_file = image_reader.read(filename_queue)
    image = tf.expand_dims(tf.image.decode_image(image_file),0)
    od_graph_def = tf.GraphDef()
    with tf.gfile.GFile(model_path, 'rb') as fid:
        serialized_graph = fid.read()
        od_graph_def.ParseFromString(serialized_graph)
        tf.import_graph_def(od_graph_def, name='',input_map={"image_tensor:0": image})
    sess = tf.InteractiveSession(graph=detection_graph)
image_tensor = detection_graph.get_tensor_by_name('image_tensor:0')
boxes = detection_graph.get_tensor_by_name('detection_boxes:0')
scores = detection_graph.get_tensor_by_name('detection_scores:0')
classes = detection_graph.get_tensor_by_name('detection_classes:0')
num_detections = detection_graph.get_tensor_by_name('num_detections:0')

In [ ]:
imname